In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [2]:
from data_utils.data_loaders import FocusCNNLoader

In [3]:
loader = FocusCNNLoader(
    batch_size=4,
    shuffle=True,
    num_workers=1,
    images_dir = "/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/images",
    csv_path="/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/labels.csv",
    labels={
        "0": "none",
        "1": "person",
        "2": "car",
        "3": "bicycle"
    },
    tf_image_size=(640, 640),
    save_out_dir="/home/ubuntu/focus-convolutional-neural-network/res/focus_cnn/coco_focuscnn/trainer/labels/",
    validation_split=0.15,
    is_test=False
)

In [4]:
dl = loader.get_train_loader()
x = next(iter(dl))
print(f"{x[0].shape=}")
print(f"{x[1].shape=}")

x[0].shape=torch.Size([4, 3, 640, 640])
x[1].shape=torch.Size([4])


# Models

In [5]:
from models.focuscnn.FocusCNN import FocusCNN
from models.focus.ResFocusNetwork import ResFocusNetwork
from models.classifier.MultiClassifier import MultiClassifier
from collections import OrderedDict

In [6]:
import torch

In [7]:
classifier = MultiClassifier(
    n_classes=4,
    backbone="resnet34",
)
classifier_path = "/home/ubuntu/focus-convolutional-neural-network/res/classifiers/coco_classifier_multi/trainer/0219_215726/models/checkpoint-epoch59.pth"

focus_networks = OrderedDict({
    "1": ResFocusNetwork(
        loss_lambda_tr=1,
        loss_lambda_rot=1,
        loss_lambda_sc=1,
        backbone="resnet34",
        threshold=0.5,
        inp_img_size=(640, 640),
        loss_rot=False
    ),
    "2": ResFocusNetwork(
        loss_lambda_tr=1,
        loss_lambda_rot=1,
        loss_lambda_sc=1,
        backbone="resnet34",
        threshold=0.5,
        inp_img_size=(640, 640),
        loss_rot=False
    ),
    "3": ResFocusNetwork(
        loss_lambda_tr=1,
        loss_lambda_rot=1,
        loss_lambda_sc=1,
        backbone="resnet34",
        threshold=0.5,
        inp_img_size=(640, 640),
        loss_rot=False
    ),
})
focus_networks_paths = OrderedDict({
    "1": "/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_person/trainer/0103_092318/models/checkpoint-epoch79.pth",
    "2": "/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_car/trainer/0111_050401/models/checkpoint-epoch19.pth",
    "3": "/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_bicycle/trainer/0114_175807/models/checkpoint-epoch56.pth"
})

In [8]:
model = FocusCNN(
    classifier_model=classifier,
    focus_models=focus_networks,
    inp_img_size=(640, 640),
    out_img_size=(300, 300)
)

In [9]:
model.load_model(
    classifier_model_path=classifier_path,
    focus_models_path=focus_networks_paths
)

In [10]:
dl_train = loader.get_train_loader()

In [11]:
loss = torch.nn.CrossEntropyLoss()

In [12]:
iter_dl = iter(dl_train)
bn_1 = next(iter_dl)
bn_2 = next(iter_dl)

In [13]:
model.train()

In [14]:
model.to("cuda")

In [15]:
data = bn_1

inp_img = data[0].to(device="cuda")

for k in data[2]['1'].keys():
    data[2]['1'][k] = data[2]['1'][k].to(device="cuda")
    data[2]['2'][k] = data[2]['2'][k].to(device="cuda")
    data[2]['3'][k] = data[2]['3'][k].to(device="cuda")

In [23]:
data[2]

{'1': {'label': tensor([0., 1., 0., 1.], device='cuda:0'),
  'transform': tensor([[ 0.0000,  0.0000,  0.0000],
          [ 0.7947,  0.2973, -2.1448],
          [ 0.0000,  0.0000,  0.0000],
          [-0.0508,  0.0064, -0.9781]], device='cuda:0')},
 '2': {'label': tensor([0., 0., 0., 2.], device='cuda:0'),
  'transform': tensor([[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.2423, -0.5398, -0.6032]], device='cuda:0')},
 '3': {'label': tensor([0., 0., 0., 0.], device='cuda:0'),
  'transform': tensor([[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]], device='cuda:0')}}

In [25]:
isinstance(data[2].keys(), dict)

False

In [16]:
out = model(x=inp_img, target=data[2])

/home/ubuntu/focus-convolutional-neural-network/src/pipeline/loss.py:113: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  * target_cls.T


In [17]:
target=data[2]
target_cls = torch.cat(
        [
            target[cls_idx]["label"]
            for cls_idx in model.focus_models.keys()
        ]
    ).type(torch.LongTensor).to(inp_img.device)


In [18]:
loss_val = loss(out["out_cls"], target_cls)
loss_val

tensor(4.2140, device='cuda:0', grad_fn=<NllLossBackward0>)

In [19]:
loss_val.backward()

In [20]:
target_cls

tensor([0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0], device='cuda:0')

In [21]:
torch.argmax(out["out_cls"], dim=1)

tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1], device='cuda:0')